# Advent of Code 2022

## Solve Time Stats

Code by [leechristie](https://github.com/leechristie).

### Imports

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import plotly.express as px
import numpy as np

### Web Requests

In [ ]:
# requests the leaderboard for a given day, there is a sleep to prevent requesting too often
def get_leaderboard_html(day: int) -> str:
    assert type(day) == int
    assert 1 <= day <= 25
    url = f'https://adventofcode.com/2022/leaderboard/day/{day}'
    print(f'requesting {url}', end='')
    for _ in range(30):
        time.sleep(1)
        print(' .', end='')
    result = requests.request(method='GET', url=url)
    status = result.status_code
    print(f' ({status})')
    if status != 200:
        raise RuntimeError(f'Request {url} returned status code {status}')
    return result.text

In [ ]:
first_day = 1
last_day = 25

html = {}

for day in range(first_day, last_day+1):
    html[day] = get_leaderboard_html(day)

In [ ]:
for day in range(first_day, last_day+1):
    print(f'day {day}: {len(html[day])} bytes')

### Parsing

In [ ]:
def get_position(entry):
    return int(entry.find(class_='leaderboard-position').text.strip(' )'))

def get_minutes(entry):
    hh, mm, ss = entry.find(class_='leaderboard-time').text.split(' ')[-1].split(':')
    hh = int(hh)
    mm = int(mm)
    ss = int(ss)
    return hh * 60 + mm + ss / 60

In [ ]:
def add_rows(day, col_days, col_var, col_places, col_time):

    soup = BeautifulSoup(html[day], 'html.parser')

    entries = soup.find_all('div', class_="leaderboard-entry")
    gold, silver = entries[:100], entries[100:]

    prev_gold = 0
    for i in range(100):

        col_days.append(str(day))
        col_var.append('gold')
        col_places.append(i+1)

        current_gold = get_minutes(gold[i])
        delta_gold = current_gold - prev_gold
        prev_gold = current_gold
        col_time.append(delta_gold)

    prev_silver = 0
    for i in range(100):

        col_days.append(str(day))
        col_var.append('silver')
        col_places.append(i+1)

        current_silver = get_minutes(silver[i])
        delta_silver = current_silver - prev_silver
        prev_silver = current_silver
        col_time.append(delta_silver)

def build_dataframe(first_day, last_day):
    col_days = []
    col_var = []
    col_places = []
    col_time = []
    for day in range(first_day, last_day + 1):
        add_rows(day, col_days, col_var, col_places, col_time)
    return pd.DataFrame(data={
        'Day': col_days,
        'Variable': col_var,
        'Place (#1 ~ #100)': col_places,
        'Time (Minutes)': col_time
    })

df = build_dataframe(first_day, last_day)

df

### Plotting

In [ ]:
px.bar(df[df['Variable']=='gold'],
       x='Day',
       y='Time (Minutes)',
       color='Place (#1 ~ #100)',
       barmode='stack',
       title='Time for 1st through 100th Person to Solve Advent of Code 2022',
       color_continuous_scale=[r'rgba(0,0,0,0)'] + [f'rgba({r},0,{255-r},1)' for r in np.linspace(0, 255, 100)],
       height=675,
       width=1200,
       log_y=False)